In [ ]:
# Imports 

import io
import os
import pickle
import statistics
import numpy as np
import pandas as pd

import cornac
from cornac.models import WMF, EASE
from cornac.metrics import MAE, RMSE, Precision, Recall, NDCG, AUC, MAP
from cornac.eval_methods import BaseMethod

from sklearn.model_selection import train_test_split


In [ ]:
# Importing File with Ground Truth

data = pd.read_csv('./recommendations_filtered.csv')

train, test = train_test_split(data, test_size = 0.2)


In [ ]:
# Define and Train Model

seed = 1234
K = 50
max_iter = 50
learning_rate = 0.01
a = 0.1
b = 0.01
lambda_u = 0.01
lambda_v = 0.01

wmf = WMF(
    
    k = K, 
    max_iter = max_iter, 
    a = a,
    b = b, 
    learning_rate = learning_rate, 
    verbose = True,
    seed = seed, 
    lambda_u = lambda_u,
    lambda_v = lambda_v
    
)

eval_metrics = [
    
  cornac.metrics.NDCG(k = 20),
  cornac.metrics.NCRR(k = 20),
  cornac.metrics.Recall(k = 20),
  cornac.metrics.AUC()
    
]

eval_method = BaseMethod.from_splits(
    
    train_data = train.values,
    test_data = test.values,
    exclude_unknowns = False,
    verbose = True
    
)

cornac.Experiment(eval_method = eval_method, models = [wmf], metrics = eval_metrics).run()


In [ ]:
# Generate Recommendations

TOPK = 50
recommendationsArray = []

item_id2idx = wmf.train_set.iid_map

items_ranked = []

for k, v in item_id2idx.items():
    
    items_ranked.append((v,k))
    
sort_items_ranked = sorted(items_ranked, key = lambda x: x[0])

user_id2idx = list(wmf.train_set.uid_map.items())

sorted_user_id2idx = sorted(user_id2idx, key = lambda x: x[0])

for i in range(len(sorted_user_id2idx)):
    
    recommendations, scores = wmf.rank(sorted_user_id2idx[i][1])
    recommendations = recommendations[:TOPK]
    recommendations = [x + 1 for x in recommendations]
    recommendationsArray.append(recommendations)


In [ ]:
# Save Recommendations

with open("output.txt", "w") as txt_file:
    
    for i in range(len(user_id2idx)):
        
        count = 0
        
        for line in recommendationsArray[i]:
            
            txt_file.write(str(line))
            
            if(count == TOPK - 1):
                
                txt_file.write("\n")
                
            else:
                
                txt_file.write(" ")
                
            count += 1
